<a href="https://colab.research.google.com/github/25maikelo/ProyectoTopicosI/blob/main/proyecto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import panel as pn
import panel.widgets as pnw
pn.extension('plotly')

import pandas as pd
import numpy as np
import plotly.graph_objects as go
import math
from scipy.stats import wrapcauchy, levy_stable

In [9]:
# Cargar dataset
dataset = pd.read_csv('mxmh_survey_results.csv')

# TODO: Limipar dataset

In [ ]:
# Functions


In [ ]:
# Graphic Example

In [25]:
def generate_bar_plot(x, title):
  trace = go.Histogram(
      x=x,
      # marker=dict(
      #     size=12,
      #     color='rgb(255,0,0)',
      #     opacity=0.5
      # )
  )
  layout = go.Layout(
      title=title,
      scene=dict(
          xaxis=dict(title=title),
          yaxis=dict(title='Frecuencia')
      )
  )
  fig = go.Figure(data=[trace], layout=layout)
  fig.update_layout(width=400, height=300)

  return fig

def generate_hist_plot(x,y, metric, trajectory_name):
    trace = go.Scatter(
        x=x,
        y=y
    )
    layout = go.Layout(
        title=f'Metrica {metric} de {trajectory_name}',
        xaxis=dict(title='X'),
        yaxis=dict(title='Y')
    )
    fig = go.Figure(data=[trace], layout=layout)
    fig.update_layout(width=400, height=300)

    return fig

def generate_disp_plot(x,y, metric, trajectory_name):
    trace = go.Scatter(
        x=x,
        y=y
    )
    layout = go.Layout(
        title=f'Metrica {metric} de {trajectory_name}',
        xaxis=dict(title='X'),
        yaxis=dict(title='Y')
    )
    fig = go.Figure(data=[trace], layout=layout)
    fig.update_layout(width=400, height=300)

    return fig

#Selectores
datasets = pnw.RadioButtonGroup(name='Data', options=['Origen', 'Personalizada'], margin=(0,0,0,0), width=200)

#NO TOMADAS EN CUENTA
#BPM
#OCD	[1,10]


#ENTEROS
#Age	[10,89]

#DECIMALES
#Anxiety	[1,10]
#Depression	[1,10]
#Insomnia	[1,10]
#Hours per day	[0,24]

#Parametros
edad = pnw.IntInput(name='Edad', start=10, end=89,step=1, width=200)

ansiedad = pnw.FloatSlider(name='Ansiedad', start=1, end=10.0, step=0.5, width=200)
depresion = pnw.FloatSlider(name='Depresión', start=1, end=10.0, step=0.5, width=200)
insomnio = pnw.FloatSlider(name='Insomnio', start=1, end=10.0, step=0.5, width=200)
horas_dia = pnw.FloatSlider(name='Horas por día', start=1, end=10.0, step=0.5, width=200)

#CATEGORICAS
#While working	[Si, No]
#Instrumentalist	[Si, No]
#Composer	[Si, No]
#Exploratory	[Si, No]
#Foreign languages [Si, No]
#Primary streaming service [Apple Music, Sin servicio, Otro, Pandora, Spotify, Youtube Music]
#Fav genre	[Clasica, Country, EDM, Folk, Gospel, Hip-Hop, Jazz, K-pop, Latin, Lo-Fi, Metal, Pop, R&B, Rap, Rock, Videjuegos]
#Music effects	(Mejoro,Empeoro,Sin cambios)
#(Nunca, Rara vez, Algunas veces, Con frecuencia)
#Frequency [Classical]	Frequency [Country]	Frequency [EDM]	Frequency [Folk]	Frequency [Gospel]	Frequency [Hip hop]	Frequency [Jazz]
#Frequency [K pop]	Frequency [Latin]	Frequency [Lofi]	Frequency [Metal]	Frequency [Pop]	Frequency [R&B]	Frequency [Rap]	Frequency [Rock]
#Frequency [Video game music]

trabajando = pnw.Select(name='Trabajando', options=['Sí', 'No']	, width=200)
instrumental = pnw.Select(name='Instrumental', options=['Sí', 'No']	, width=200)
compositor = pnw.Select(name='Compositor', options=['Sí', 'No']	, width=200)
exploracion = pnw.Select(name='Exploración', options=['Sí', 'No']	, width=200)
extranjero = pnw.Select(name='Extranjero', options=['Sí', 'No']	, width=200)
servicio_streaming = pnw.Select(name='Servicio Striming', options=['Apple Music', 'Sin servicio', 'Otro', 'Pandora', 'Spotify', 'Youtube Music']	, width=200)
genero = pnw.Select(name='Género', options=['Clasica', 'Country', 'EDM', 'Folk', 'Gospel', 'Hip-Hop', 'Jazz', 'K-pop', 'Latin', 'Lo-Fi', 'Metal',
                                            'Pop', 'R&B', 'Rap', 'Rock', 'Videjuegos']	, width=200)
frecuencia = pnw.Select(name='Frecuencia', options=['Nunca', 'Rara vez', 'Algunas veces', 'Con frecuencia'])

# Graficas
bar_plot = pn.pane.Plotly()
hist_plot = pn.pane.Plotly()
disp_plot = pn.pane.Plotly()

dynamic_widgets_1 = pn.Row()
dynamic_widgets_2 = pn.Row()

def update_widgets_1(datasets):
  if datasets == 'Origen':
    return []
  else:
    return [edad,ansiedad,depresion,insomnio,horas_dia,trabajando]

def update_widgets_2(datasets):
  if datasets == 'Origen':
    return []
  else:
    return [instrumental,compositor,exploracion,extranjero,servicio_streaming,genero,frecuencia]

def update_plots():
  edad = edad.value,ansiedad.value,depresion.value,insomnio.value,horas_dia.value,trabajando.value,instrumental.value,compositor.value,exploracion.value,extranjero.value,servicio_streaming.value,genero.value,frecuencia.value
  return

In [6]:
#Dependencias

#Cambio de trayectoria
@pn.depends(datasets.param.value)
def parameters_1(datasets):
  dynamic_widgets_1.clear()
  widgets = update_widgets_1(datasets)
  if widgets:
      dynamic_widgets_1.extend(widgets)
  return dynamic_widgets_1

@pn.depends(datasets.param.value)
def parameters_2(datasets):
  dynamic_widgets_2.clear()
  widgets = update_widgets_2(datasets)
  if widgets:
      dynamic_widgets_2.extend(widgets)
  return dynamic_widgets_2

#Cambio de parametros
@pn.depends(datasets.param.value,edad.param.value,ansiedad.param.value,depresion.param.value,insomnio.param.value,horas_dia.param.value,trabajando.param.value,instrumental.param.value,compositor.param.value,exploracion.param.value,extranjero.param.value,servicio_streaming.param.value,genero.param.value,frecuencia.param.value)
def check_all_widgets(*args):
  update_plots()

In [7]:
pn.config.sizing_mode = 'stretch_width'  # Para limitar la expansión horizontal

# Crea tu dashboard aquí
dashboard = pn.Column(
    pn.Row(datasets),
    parameters_1,
    parameters_2,
    pn.Row(bar_plot, hist_plot, disp_plot)
)

pn.config.max_width = 300  # Establece el ancho máximo
pn.config.max_height = 300  # Establece la altura máxima

dashboard.servable(title='Dashboard')

Column(sizing_mode='stretch_width')
    [0] Row(sizing_mode='stretch_width')
        [0] RadioButtonGroup(margin=(0, 0, 0, 0), name='Data', options=['Origen', 'Personalizada'...], value='Origen', width=200)
    [1] ParamFunction(function, _pane=Row, defer_load=False, sizing_mode='stretch_width')
    [2] ParamFunction(function, _pane=Row, defer_load=False, sizing_mode='stretch_width')
    [3] Row(sizing_mode='stretch_width')
        [0] Plotly(None)
        [1] Plotly(None)
        [2] Plotly(None)